**Preparation**

In [154]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data_dir = '/content/drive/MyDrive/BioInformatics/BiologicalData_project/data'

In [156]:
!pip install biopython

In [157]:
from Bio import SeqIO
from Bio.Blast import NCBIXML
import pandas as pd
import requests

# Generating the MSA


### Step 1 -  Starting from the BLAST pairwise alignments processing the provided XML file.


To generate the provided XML File, we searched the UNiref50 Database for up to 1000 Alignments and AlignViews BLASTXML.
Our Protein Sequence has the UniProt Id: Q12723 and the Pfam Id: Pfam03060

In [158]:
# Parse the XML output
# https://www.biostars.org/p/82514/

input_file = '{}/O43099_Uniref50_1000.xml'.format(data_dir)

with open(input_file) as f:
  blast_records = NCBIXML.parse(f)
  data = []

  # Iterate PSIBLAST rounds (here just one since it is a simple BLAST)
  for blast_record in blast_records:
      query_id = blast_record.query

      # Iterate alignments objects
      for i, alignment in enumerate(blast_record.alignments):
          subject_id = alignment.title

          # Iterate pairwise alignments (High scoring pairs - HPS)
          for hsp in alignment.hsps:
              data.append((query_id,
                              subject_id,
                              blast_record.query_length,
                              hsp.query,
                              hsp.match,
                              hsp.sbjct,
                              hsp.query_start,
                              hsp.query_end,
                              hsp.sbjct_start,
                              hsp.sbjct_end,
                              hsp.identities,
                              hsp.positives,
                              hsp.gaps,
                              hsp.expect,
                              hsp.score))

              # Skip duplicated subjects
              break

# Build a dataframe
df = pd.DataFrame(data, columns=["query_id", "subject_id", "query_len",
                                  "query_seq", "match_seq", "subject_seq",
                                "query_start", "query_end", "subject_start", "subject_end",
                                "identity", "positive", "gaps", "eval", "bit_score"])
# Print an example
len(df)
df.iloc[-1]

,999
query_id,EMBOSS_001
subject_id,UR50:UniRef50_A0A804M4U7 glutaredoxin-dependen...
query_len,161
query_seq,KKVILFALPGAFTPVCSARHVPEYIEKLPEIRAKGVDVVAVLAYNDA
match_seq,KKV+LFA+ G F P C+ +H+ + K+ E AKG+D VA + NDA
subject_seq,KKVVLFAMSGTFMPTCTHKHLLGSMVKVGEFHAKGIDTVACVPVNDA
query_start,42
query_end,88
subject_start,98
subject_end,144


In [159]:
# Extract the sequence of aligned subject proteins (hits) and create the .fasta file
# Filter hits based on an e-value threshold
#We dont create the further produced MSA here yet

out_file = '{}/O43099_Uniref50_1000.fasta'.format(data_dir)

# set your query header + sequence here
query_id = "O43099_TIR"
query_seq = "KAGDSFPSDVVFSYIPWSEDKGEITACGIPINYNASKEWADKKVILFALPGAFTPVCSARHVPEYIEKLPEIRAKGVDVVAVLAYNDAYVMSAWGKANQVTGDDILFLSDPDARFSKSIGWADEEGRTKRYALVIDHGKITYAALEPAKNHLEFSSAETVL"

with open(out_file, "w") as fout:
    # 1) write query/reference first
    fout.write(f">{query_id}\n{query_seq}\n")

    # 2) write mapped BLAST hits (query-length with gaps)
    for index, row in df.iterrows():
        if row["eval"] < 0.001:
            mapped_seq = ["-"] * blast_record.query_length
            c = 0
            for l_q, l_s in zip(row['query_seq'], row['subject_seq']):
                if l_q != " " and l_q != '-':
                    mapped_seq[row["query_start"] + c - 1] = l_s if l_s != " " else "-"
                    c += 1
            fout.write(">{}\n{}\n".format(row["subject_id"], "".join(mapped_seq)))

print("Wrote combined FASTA (query first):", out_file)
print("Total rows in df:", len(df))

Wrote combined FASTA (query first): /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_Uniref50_1000.fasta
Total rows in df: 1000


In [160]:
#Preparation Step- Build a fasta with only our given rpotein sequence
query_file = f"{data_dir}/O43099_reference_query.fasta"

with open(query_file, "w") as f:
    f.write(">O43099_TIR\n")
    f.write("KAGDSFPSDVVFSYIPWSEDKGEITACGIPINYNASKEWADKKVILFALPGAFTPVCSARHVPEYIEKLPEIRAKGVDVVAVLAYNDAYVMSAWGKANQVTGDDILFLSDPDARFSKSIGWADEEGRTKRYALVIDHGKITYAALEPAKNHLEFSSAETVL\n")

In [161]:
#Build MSA from that joined fasta
# Install clustalo
!apt-get -qq update
!apt-get -qq install -y clustalo

msa_raw = f"{data_dir}/O43099_raw_msa.fasta"



# --output-order=input-order keeps query first
!clustalo -i "{out_file}" -o "{msa_raw}" --force --outfmt=fasta --output-order=input-order

print("MSA written:", msa_raw)


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
MSA written: /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_raw_msa.fasta


In [162]:
#Clean that new MSA
!pip -q install biopython

from Bio import SeqIO
from collections import Counter
import re, os

msa_in  = msa_raw
msa_out = f"{data_dir}/O43099_final_msa.fasta"

MAX_GAP_FRAC_PER_SEQ = 0.60
MAX_GAP_FRAC_PER_COL = 0.50
DEDUPLICATE_SEQS     = True

ALLOWED = set(list("ACDEFGHIKLMNPQRSTVWYXBZUOJU-"))

def normalize_seq(s: str) -> str:
    s = s.upper().replace(".", "-")
    s = re.sub(r"\s+", "", s)
    s = "".join((c if c in ALLOWED else "X") for c in s)
    return s

records = list(SeqIO.parse(msa_in, "fasta"))
if not records:
    raise ValueError(f"No sequences found in {msa_in}")

# normalize + enforce equal length
for r in records:
    r.seq = type(r.seq)(normalize_seq(str(r.seq)))

L = max(len(r) for r in records)
for r in records:
    s = str(r.seq)
    if len(s) < L:
        s += "-" * (L - len(s))
    elif len(s) > L:
        s = s[:L]
    r.seq = type(r.seq)(s)

def gap_frac(seq: str) -> float:
    return seq.count("-") / len(seq)

# keep query (first record) even if gappy; filter others
filtered = [records[0]] + [r for r in records[1:] if gap_frac(str(r.seq)) <= MAX_GAP_FRAC_PER_SEQ]
records = filtered

# deduplicate identical aligned sequences
if DEDUPLICATE_SEQS:
    seen = set()
    dedup = []
    for r in records:
        s = str(r.seq)
        if s not in seen:
            seen.add(s)
            dedup.append(r)
    records = dedup

seqs = [str(r.seq) for r in records]
L = len(seqs[0])

# column mask (gap filter)
keep_mask = []
for col in range(L):
    colchars = [s[col] for s in seqs]
    gap_f = colchars.count("-") / len(colchars)
    keep_mask.append(gap_f <= MAX_GAP_FRAC_PER_COL)

kept_cols = [i for i, k in enumerate(keep_mask) if k]
if len(kept_cols) < 10:
    raise ValueError("Too many columns removed — relax MAX_GAP_FRAC_PER_COL.")

def apply_mask(seq: str, cols):
    return "".join(seq[i] for i in cols)

for r in records:
    r.seq = type(r.seq)(apply_mask(str(r.seq), kept_cols))

os.makedirs(os.path.dirname(msa_out), exist_ok=True)
SeqIO.write(records, msa_out, "fasta")

print("Cleaned MSA written:", msa_out)
print("Sequences:", len(records))
print("Alignment length:", len(records[0].seq))
print("Query first:", records[0].id)


Cleaned MSA written: /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_final_msa.fasta
Sequences: 920
Alignment length: 125
Query first: O43099_TIR


---
# **Model Construction**

**Building The HMM Model**

In [163]:
#Building the HMM
!apt-get update -qq
!apt-get install -y hmmer

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
hmmer is already the newest version (3.3.2+dfsg-1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.


In [164]:
!hmmbuild -h | head
!hmmsearch -h | head

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Usage: hmmbuild [-options] <hmmfile_out> <msafile>

Basic options:
  -h     : show brief help on version and usage
  -n <s> : name the HMM <s>
# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
Usage: hmmsearch [options] <hmmfile> <seqdb>

Basic options:
  -h : show brief help on version and usage



In [165]:
!apt-get install -y ncbi-blast+
!psiblast -h | head

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ncbi-blast+ is already the newest version (2.12.0+ds-3build1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.
USAGE
  psiblast [-h] [-help] [-import_search_strategy filename]
    [-export_search_strategy filename] [-db database_name]
    [-dbsize num_letters] [-gilist filename] [-seqidlist filename]
    [-negative_gilist filename] [-negative_seqidlist filename]
    [-taxids taxids] [-negative_taxids taxids] [-taxidlist filename]
    [-negative_taxidlist filename] [-ipglist filename]
    [-negative_ipglist filename] [-entrez_query entrez_query]
    [-subject subject_input_file] [-subject_loc range] [-query input_file]
    [-out output_file] [-evalue evalue] [-word_size int_value]


In [166]:
!apt-get update -qq
!apt-get install -y ncbi-blast+

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ncbi-blast+ is already the newest version (2.12.0+ds-3build1).
0 upgraded, 0 newly installed, 0 to remove and 80 not upgraded.


In [167]:
!hmmbuild "{data_dir}/O43099_Uniref50_1000.hmm" "{data_dir}/O43099_final_msa.fasta"

# hmmbuild :: profile HMM construction from multiple sequence alignments
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# input alignment file:             /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_final_msa.fasta
# output HMM file:                  /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_Uniref50_1000.hmm
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

# idx name                  nseq  alen  mlen eff_nseq re/pos description
#---- -------------------- ----- ----- ----- -------- ------ -----------
1     O43099_final_msa       920   125   124     3.78  0.590 

# CPU time: 0.10u 0.00s 00:00:00.10 Elapsed: 00:00:00.11


**Building the PSSM Model**

In [168]:
db_fasta = f"{data_dir}/protFamily_db_ungapped.fasta"
db_name  = f"{data_dir}/blastdb/protFamily_db_ungapped"

# make ungapped DB sequences (BLAST DB should not contain '-')
recs = []
for rec in SeqIO.parse(out_file, "fasta"):
    rec.seq = type(rec.seq)(str(rec.seq).replace("-", ""))
    recs.append(rec)

SeqIO.write(recs, db_fasta, "fasta")

!makeblastdb -in "{db_fasta}" -dbtype prot -out "{db_name}"
print("BLAST DB built:", db_name)




Building a new DB, current time: 02/19/2026 15:48:38
New DB name:   /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/blastdb/protFamily_db_ungapped
New DB title:  /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/protFamily_db_ungapped.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/blastdb/protFamily_db_ungapped
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 1001 sequences in 0.128257 seconds.


BLAST DB built: /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/blastdb/protFamily_db_ungapped


In [169]:
#Build final pssm
clean_msa = f"{data_dir}/O43099_final_msa.fasta"
pssm_out  = f"{data_dir}/O43099_Uniref50_1000.pssm"
log_out   = f"{data_dir}/O43099_pssm.log"

!psiblast \
  -in_msa "{clean_msa}" \
  -db "{db_name}" \
  -num_iterations 1 \
  -out_pssm "{pssm_out}" \
  -out "{log_out}"

---
**Generating Model Predictions against SwissProt**


PSSM in Swissprot

In [170]:
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
!gunzip uniprot_sprot.fasta.gz
!makeblastdb -in uniprot_sprot.fasta -dbtype prot

--2026-02-19 15:48:39--  https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.fasta.gz
Resolving ftp.uniprot.org (ftp.uniprot.org)... 128.175.240.195
Connecting to ftp.uniprot.org (ftp.uniprot.org)|128.175.240.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93457057 (89M) [application/x-gzip]
Saving to: ‘uniprot_sprot.fasta.gz’

uniprot_sprot.fasta 100%[===================>]  89.13M  38.9MB/s    in 2.3s    

2026-02-19 15:48:41 (38.9 MB/s) - ‘uniprot_sprot.fasta.gz’ saved [93457057/93457057]

gzip: uniprot_sprot.fasta already exists; do you wish to overwrite (y or n)? y


Building a new DB, current time: 02/19/2026 15:49:23
New DB name:   /content/uniprot_sprot.fasta
New DB title:  uniprot_sprot.fasta
Sequence type: Protein
Deleted existing Protein BLAST database named /content/uniprot_sprot.fasta
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 574627 sequences in 18.6091 seconds

In [171]:
#Here we run hmm against swiss prot
#tblout for protein matching
#domtblout for position matching
!hmmsearch \
  --tblout "{data_dir}/O43099_hmm_vs_swissprot.tbl" \
  --domtblout "{data_dir}/O43099_hmm_vs_swissprot.domtbl" \
  "{data_dir}/O43099_Uniref50_1000.hmm"\
  uniprot_sprot.fasta

# hmmsearch :: search profile(s) against a sequence database
# HMMER 3.3.2 (Nov 2020); http://hmmer.org/
# Copyright (C) 2020 Howard Hughes Medical Institute.
# Freely distributed under the BSD open source license.
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -
# query HMM file:                  /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_Uniref50_1000.hmm
# target sequence database:        uniprot_sprot.fasta
# per-seq hits tabular output:     /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_hmm_vs_swissprot.tbl
# per-dom hits tabular output:     /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_hmm_vs_swissprot.domtbl
# - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - -

Query:       O43099_final_msa  [M=124]
Scores for complete sequences (score includes all domains):
   --- full sequence ---   --- best 1 domain ---    -#dom-
    E-value  score  bias    E-value 

In [172]:
#Here we collect the nessecary columns sstart/send to get the proteins position
!psiblast \
  -in_pssm "{data_dir}/O43099_Uniref50_1000.pssm" \
  -db uniprot_sprot.fasta \
  -outfmt "6 sseqid qstart qend sstart send evalue bitscore length pident" \
  -out "{data_dir}/O43099_pssm_vs_swissprot.tsv"

Summary of what we have now:

Collect the list of retrieved hits:
pssm -> sseqid in TSV "Q12723_pssm_vs_swissprot.tsv"
hmm- ->  targets in domtbl

Collect matching positions in retrieved hits
pssm -> sstart/send
hmm -> domtbl Koordinaten "Q12723_hmm_vs_swissprot.domtbl"

**Search Results Ground Truth - From our Pfam Id**


In [173]:
#Now we check for our Pfam Annotations in the swiss prot database, from the provided TSV with only reviewd proteins
# We also pritn out all annotation for later calculations.
import pandas as pd

path = f"{data_dir}/protein2ipr_sprot.dat"
df = pd.read_csv(path, sep="\t", header=None)

pfam_id = "PF08534"

# All proteins in SwissProt annotation file
all_swissprot = set(df[0])

# Ground truth proteins for this Pfam
ground_truth = set(df[df[3] == pfam_id][0])

print("Total SwissProt proteins:", len(all_swissprot))
print("Ground truth proteins (PF08534):", len(ground_truth))



Total SwissProt proteins: 534419
Ground truth proteins (PF08534): 110


**Search Result Processing - pssm**

In [174]:
import pandas as pd

pssm_path = f"{data_dir}/O43099_pssm_vs_swissprot.tsv"

pssm_df = pd.read_csv(
    pssm_path,
    sep="\t",
    header=None,
    names=["sseqid","qstart","qend","sstart","send","evalue","bitscore","length","pident"]
)

# extract UniProt accession from sseqid
# handles: sp|Q9XXXX|NAME  -> Q9XXXX
# also leaves Q9XXXX unchanged if it's already just an accession
pssm_df["accession"] = pssm_df["sseqid"].str.split("|").str[1].fillna(pssm_df["sseqid"])

print(pssm_df.head(10))
print("Unique accessions in PSSM hits:", pssm_df["accession"].nunique())

                  sseqid  qstart  qend  sstart  send        evalue  bitscore  \
0   sp|O43099|PRX5_ASPFU       1   125      41   165  1.160000e-57     179.0   
1   sp|Q5ASN8|PRX5_EMENI       1   125      41   165  1.370000e-56     176.0   
2   sp|Q9Y8B8|PRX5_PENCI       1   125      40   164  8.980000e-55     171.0   
3   sp|A9PCL4|PRX2_POPTR       1   125      31   159  2.200000e-52     165.0   
4  sp|P30044|PRDX5_HUMAN       1   125      80   211  7.200000e-52     166.0   
5  sp|Q9FR35|PRX2C_ORYSJ       2   125      32   159  8.720000e-52     163.0   
6  sp|Q9XEX2|PRX2B_ARATH       1   125      31   159  9.520000e-52     163.0   
7  sp|Q7F8S5|PR2E2_ORYSJ       1   125      91   222  1.410000e-51     165.0   
8  sp|O22711|PRX2D_ARATH       4   125      34   159  2.660000e-51     162.0   
9  sp|Q9BGI1|PRDX5_BOVIN       1   125      85   216  3.090000e-51     164.0   

   length   pident accession  
0     125  100.000    O43099  
1     125   91.200    Q5ASN8  
2     125   83.200    Q9Y8

**Search result Processing - hmm**

In [175]:
# now exctracting hmm hits
import pandas as pd

hmm_path = f"{data_dir}/O43099_hmm_vs_swissprot.domtbl"

rows = []
with open(hmm_path, "r") as f:
    for line in f:
        if line.startswith("#") or not line.strip():
            continue
        parts = line.strip().split()
        rows.append(parts[:22])  # fixed domtbl columns

hmm_df = pd.DataFrame(rows)

# column 0 = target name (often like sp|Q9XXXX|NAME)
hmm_df = hmm_df.rename(columns={0: "tname"})

# extract UniProt accession
hmm_df["accession"] = hmm_df["tname"].str.split("|").str[1].fillna(hmm_df["tname"])

print(hmm_df[["tname", "accession"]].head(10))
print("Unique accessions in HMM hits:", hmm_df["accession"].nunique())

                   tname accession
0   sp|O43099|PRX5_ASPFU    O43099
1   sp|Q5ASN8|PRX5_EMENI    Q5ASN8
2   sp|Q9Y8B8|PRX5_PENCI    Q9Y8B8
3  sp|Q9XEX2|PRX2B_ARATH    Q9XEX2
4  sp|Q69TY4|PR2E1_ORYSJ    Q69TY4
5   sp|A9PCL4|PRX2_POPTR    A9PCL4
6  sp|O22711|PRX2D_ARATH    O22711
7  sp|Q7F8S5|PR2E2_ORYSJ    Q7F8S5
8  sp|Q9FR35|PRX2C_ORYSJ    Q9FR35
9  sp|Q949U7|PRX2E_ARATH    Q949U7
Unique accessions in HMM hits: 226


## **Comparing Ground Truth with our models - Protein Level**

Ground Truth Set up

In [176]:
#Setting ground truth at protein level
pfam_id = "PF08534"

all_swissprot_proteins = set(df[0])

gt_proteins = set(df.loc[df[3] == pfam_id, 0])

print("Total SwissProt proteins:", len(all_swissprot_proteins))
print(f"Ground truth proteins ({pfam_id}):", len(gt_proteins))


Total SwissProt proteins: 534419
Ground truth proteins (PF08534): 110


In [177]:
#Transforming MOdel Predictions to protein sets
# Model predictions at PROTEIN level
pssm_proteins = set(pssm_df["accession"])
hmm_proteins  = set(hmm_df["accession"])

print("Unique PSSM proteins:", len(pssm_proteins))
print("Unique HMM proteins:", len(hmm_proteins))


Unique PSSM proteins: 211
Unique HMM proteins: 226


In [178]:
#Setting up Confusion Matrix
def protein_level_confusion(pred_proteins, gt_proteins, universe_proteins):
    TP = len(pred_proteins & gt_proteins)
    FP = len(pred_proteins - gt_proteins)
    FN = len(gt_proteins - pred_proteins)
    TN = len(universe_proteins - (pred_proteins | gt_proteins))
    return TP, FP, FN, TN


Compute Confusion mAtrices for hmm and pssm

In [179]:
TP_pssm, FP_pssm, FN_pssm, TN_pssm = protein_level_confusion(
    pssm_proteins, gt_proteins, all_swissprot_proteins
)

print("PSSM Confusion Matrix (Protein Level)")
print("TP:", TP_pssm, "FP:", FP_pssm, "FN:", FN_pssm, "TN:", TN_pssm)

PSSM Confusion Matrix (Protein Level)
TP: 52 FP: 159 FN: 58 TN: 534153


In [180]:
TP_hmm, FP_hmm, FN_hmm, TN_hmm = protein_level_confusion(
    hmm_proteins, gt_proteins, all_swissprot_proteins
)

print("HMM Confusion Matrix (Protein Level)")
print("TP:", TP_hmm, "FP:", FP_hmm, "FN:", FN_hmm, "TN:", TN_hmm)

HMM Confusion Matrix (Protein Level)
TP: 56 FP: 170 FN: 54 TN: 534146


Precision Matrices first set up and then in the next cell computed

In [181]:
import math

def compute_metrics(TP, FP, FN, TN):

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0

    f1 = (2 * precision * recall / (precision + recall)
          if (precision + recall) > 0 else 0)

    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
    balanced_accuracy = (recall + specificity) / 2

    denominator = math.sqrt((TP + FP)*(TP + FN)*(TN + FP)*(TN + FN))
    mcc = ((TP * TN - FP * FN) / denominator
           if denominator > 0 else 0)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Balanced Accuracy": balanced_accuracy,
        "MCC": mcc
    }


In [182]:
pssm_metrics = compute_metrics(TP_pssm, FP_pssm, FN_pssm, TN_pssm)

print("PSSM – Protein Level Metrics")
for k, v in pssm_metrics.items():
    print(f"{k}: {v:.4f}")


PSSM – Protein Level Metrics
Precision: 0.2464
Recall: 0.4727
F1-score: 0.3240
Balanced Accuracy: 0.7362
MCC: 0.3411


In [183]:
hmm_metrics = compute_metrics(TP_hmm, FP_hmm, FN_hmm, TN_hmm)

print("HMM – Protein Level Metrics")
for k, v in hmm_metrics.items():
    print(f"{k}: {v:.4f}")


HMM – Protein Level Metrics
Precision: 0.2478
Recall: 0.5091
F1-score: 0.3333
Balanced Accuracy: 0.7544
MCC: 0.3550


# **Comparing the Models - Domain Level**

In [184]:
# Pfam (ground truth) domains with start/end
import pandas as pd

# df is already loaded from protein2ipr_sprot.dat in your notebook
# pfam_id is already set (e.g., "PF08534")

pfam_rows = df[df[3] == pfam_id].copy()

# auto-detect start/end columns (take the last two numeric-like columns)
num_cols = []
for c in pfam_rows.columns:
    col = pd.to_numeric(pfam_rows[c], errors="coerce")
    if col.notna().mean() > 0.9:
        num_cols.append(c)

start_col, end_col = num_cols[-2], num_cols[-1]

pfam_domains = pfam_rows[[0, start_col, end_col]].copy()
pfam_domains.columns = ["accession", "pfam_start", "pfam_end"]

pfam_domains["pfam_start"] = pfam_domains["pfam_start"].astype(int)
pfam_domains["pfam_end"]   = pfam_domains["pfam_end"].astype(int)

print(pfam_domains.head())
print("Pfam domain rows:", len(pfam_domains))
print("Pfam proteins (residue-level GT):", pfam_domains["accession"].nunique())


       accession  pfam_start  pfam_end
71579     A5F3A2          18       161
135409    A9PCL4           6       159
183673    B3EWI1           6       162
299149    O14313           4       153
301779    O22711           6       159
Pfam domain rows: 110
Pfam proteins (residue-level GT): 110


In [185]:
# compute protein length
from Bio import SeqIO

protein_lengths = {}
for rec in SeqIO.parse("uniprot_sprot.fasta", "fasta"):
    parts = rec.id.split("|")
    acc = parts[1] if len(parts) > 1 else rec.id
    protein_lengths[acc] = len(rec.seq)

print("Protein lengths loaded:", len(protein_lengths))



Protein lengths loaded: 574627


In [186]:
# compute the residue set from pfam for the true negative clalculation
pfam_residues = {}
for _, row in pfam_domains.iterrows():
    acc = row["accession"]
    lo, hi = sorted((int(row["pfam_start"]), int(row["pfam_end"])))
    pfam_residues.setdefault(acc, set()).update(range(lo, hi + 1))

print("Pfam residue proteins:", len(pfam_residues))


Pfam residue proteins: 110


Extracting PSSM Residues


In [187]:
# build residue-level predictions for PSSM
# Predictions (PSSM): union if multiple alignments per protein
pssm_domains = pssm_df.copy()

pssm_residues = {}
for _, row in pssm_domains.iterrows():
    acc = row["accession"]
    lo, hi = sorted((int(row["sstart"]), int(row["send"])))
    pssm_residues.setdefault(acc, set()).update(range(lo, hi + 1))

print("PSSM residue proteins:", len(pssm_residues))

# inspect the same example protein if present
acc = "A5F3A2"
print("Has PSSM prediction for A5F3A2?", acc in pssm_residues)
if acc in pssm_residues:
    print("Number of PSSM predicted residues:", len(pssm_residues[acc]))
    print("First 10 predicted residues:", list(sorted(pssm_residues[acc]))[:10])
    print("Last 10 predicted residues:", list(sorted(pssm_residues[acc]))[-10:])


PSSM residue proteins: 211
Has PSSM prediction for A5F3A2? True
Number of PSSM predicted residues: 24
First 10 predicted residues: [39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
Last 10 predicted residues: [53, 54, 55, 56, 57, 58, 59, 60, 61, 62]


Extracting hmm Residues

In [188]:
# build residue-level predictions for hmm
# assign proper domtblout column names


domtbl_cols = [
    "tname","taccession","tlen",
    "qname","qaccession","qlen",
    "full_evalue","full_score","full_bias",
    "dom_num","dom_of",
    "c_evalue","i_evalue","dom_score","dom_bias",
    "hmm_from","hmm_to",
    "ali_from","ali_to",
    "env_from","env_to",
    "acc"
]

# rename numeric columns (1..21) -> the expected domtblout names excluding tname
rename_map = {i: domtbl_cols[i] for i in range(1, 22)}  # 1..21
hmm_df = hmm_df.rename(columns=rename_map)

# ensure accession exists (you already created it earlier; keep it)
if "accession" not in hmm_df.columns:
    hmm_df["accession"] = hmm_df["tname"].astype(str)

print("hmm_df columns after renaming:")
print(list(hmm_df.columns))

# now build residue sets using ali_from/ali_to (target alignment coords)
hmm_residues = {}
for _, row in hmm_df.iterrows():
    acc = row["accession"]
    lo, hi = sorted((int(row["ali_from"]), int(row["ali_to"])))
    hmm_residues.setdefault(acc, set()).update(range(lo, hi + 1))

print("HMM residue proteins:", len(hmm_residues))


hmm_df columns after renaming:
['tname', 'taccession', 'tlen', 'qname', 'qaccession', 'qlen', 'full_evalue', 'full_score', 'full_bias', 'dom_num', 'dom_of', 'c_evalue', 'i_evalue', 'dom_score', 'dom_bias', 'hmm_from', 'hmm_to', 'ali_from', 'ali_to', 'env_from', 'env_to', 'acc', 'accession']
HMM residue proteins: 226


PSSM - Residue Level Confusion Matrix - Domain Level

In [189]:
#pssm confusion matrix
TP_res_pssm = FP_res_pssm = FN_res_pssm = TN_res_pssm = 0

for acc, pfam_set in pfam_residues.items():
    pred_set = pssm_residues.get(acc, set())

    TP = len(pfam_set & pred_set)
    FP = len(pred_set - pfam_set)
    FN = len(pfam_set - pred_set)

    L = protein_lengths.get(acc)
    if L is None:
        continue

    TN = L - (TP + FP + FN)
    if TN < 0:
        continue

    TP_res_pssm += TP
    FP_res_pssm += FP
    FN_res_pssm += FN
    TN_res_pssm += TN


print("PSSM Confusion Matrix (Residue Level)")
print("TP:", TP_res_pssm,
      "FP:", FP_res_pssm,
      "FN:", FN_res_pssm,
      "TN:", TN_res_pssm)

PSSM Confusion Matrix (Residue Level)
TP: 6106 FP: 168 FN: 9190 TN: 5444


In [190]:
#HMM – Residue Level Confusion Matrice

TP_res_hmm = FP_res_hmm = FN_res_hmm = TN_res_hmm = 0

for acc, pfam_set in pfam_residues.items():
    pred_set = hmm_residues.get(acc, set())

    TP = len(pfam_set & pred_set)
    FP = len(pred_set - pfam_set)
    FN = len(pfam_set - pred_set)

    L = protein_lengths.get(acc)
    if L is None:
        continue

    TN = L - (TP + FP + FN)
    if TN < 0:
        continue

    TP_res_hmm += TP
    FP_res_hmm += FP
    FN_res_hmm += FN
    TN_res_hmm += TN


print("HMM Confusion Matrix (Residue Level)")
print("TP:", TP_res_hmm,
      "FP:", FP_res_hmm,
      "FN:", FN_res_hmm,
      "TN:", TN_res_hmm)


HMM Confusion Matrix (Residue Level)
TP: 6242 FP: 108 FN: 9054 TN: 5504


In [191]:
#Precision Set up generic
import math

def compute_metrics(TP, FP, FN, TN):
    precision = TP / (TP + FP) if (TP + FP) else 0.0
    recall    = TP / (TP + FN) if (TP + FN) else 0.0
    f1        = (2 * precision * recall / (precision + recall)) if (precision + recall) else 0.0
    bal_acc   = 0.5 * (
        (TP / (TP + FN) if (TP + FN) else 0.0) +
        (TN / (TN + FP) if (TN + FP) else 0.0)
    )
    denom = (TP + FP) * (TP + FN) * (TN + FP) * (TN + FN)
    mcc = ((TP * TN - FP * FN) / math.sqrt(denom)) if denom else 0.0
    return {"Precision": precision, "Recall": recall, "F1-score": f1, "Balanced Accuracy": bal_acc, "MCC": mcc}


In [192]:
#Precision Calculation for both models
pssm_res_metrics = compute_metrics(TP_res_pssm, FP_res_pssm, FN_res_pssm, TN_res_pssm)
hmm_res_metrics  = compute_metrics(TP_res_hmm,  FP_res_hmm,  FN_res_hmm,  TN_res_hmm)

print("PSSM – Residue Level Metrics")
for k, v in pssm_res_metrics.items():
    print(f"{k}: {v:.4f}")

print("\nHMM – Residue Level Metrics")
for k, v in hmm_res_metrics.items():
    print(f"{k}: {v:.4f}")


PSSM – Residue Level Metrics
Precision: 0.9732
Recall: 0.3992
F1-score: 0.5662
Balanced Accuracy: 0.6846
MCC: 0.3570

HMM – Residue Level Metrics
Precision: 0.9830
Recall: 0.4081
F1-score: 0.5767
Balanced Accuracy: 0.6944
MCC: 0.3747


# **Retreive the Family Sequence**

In [193]:
from Bio import SeqIO

family_sequences = list(SeqIO.parse(out_file, "fasta"))

print("Loaded sequences:", len(family_sequences))
print("Query ID:", family_sequences[0].id)

Loaded sequences: 1001
Query ID: O43099_TIR


In [194]:
import pandas as pd

# assumes family_sequences is already loaded from Step 2
candidates = pd.DataFrame({
    "seq_id": [r.id for r in family_sequences],
    "length": [len(r.seq) for r in family_sequences],
})

print("Candidates:", len(candidates))
print(candidates.head(5))


Candidates: 1001
                     seq_id  length
0                O43099_TIR     161
1      UR50:UniRef50_O43099     161
2  UR50:UniRef50_A0A364KN34     161
3  UR50:UniRef50_A0A1Q5Q9Y5     161
4  UR50:UniRef50_A0A5N6Y274     161


In [195]:
import re
import pandas as pd

# 1) Extract a UniProt-like accession from your seq_id (works for: UR50:UniRef50_A0A..., UR50:UniRef50_O43099, O43099_TIR)
def extract_accession(seq_id: str) -> str:
    # grab last token that looks like a UniProt accession-ish string
    m = re.findall(r"[A-NR-Z0-9]{1,3}[0-9][A-Z0-9]{3}[0-9]|[OPQ][0-9][A-Z0-9]{3}[0-9]", seq_id)
    return m[-1] if m else None

candidates["accession"] = candidates["seq_id"].apply(extract_accession)

print("Missing accessions:", candidates["accession"].isna().sum())
print(candidates.head(5))

# 2) Load SwissProt Pfam annotations (the file your notebook already uses)
path = f"{data_dir}/protein2ipr_sprot.dat"
df = pd.read_csv(path, sep="\t", header=None)

pfam_id = "PF08534"   # <-- this is the Pfam ground-truth used in your notebook
pfam_hits = df[df[3] == pfam_id]
ground_truth_proteins = set(pfam_hits[0])

print("Ground-truth proteins (SwissProt) for", pfam_id, ":", len(ground_truth_proteins))

# 3) How many of your 1001 candidates are in Pfam ground truth?
n_gt_in_candidates = candidates["accession"].isin(ground_truth_proteins).sum()
print("Candidates matching Pfam ground truth:", n_gt_in_candidates, "out of", len(candidates))


Missing accessions: 2
                     seq_id  length accession
0                O43099_TIR     161    O43099
1      UR50:UniRef50_O43099     161    O43099
2  UR50:UniRef50_A0A364KN34     161    A0A364
3  UR50:UniRef50_A0A1Q5Q9Y5     161  A0A1Q5Q9
4  UR50:UniRef50_A0A5N6Y274     161  A0A5N6Y2
Ground-truth proteins (SwissProt) for PF08534 : 110
Candidates matching Pfam ground truth: 20 out of 1001


In [196]:
import os, glob
import pandas as pd

# 1) Look for likely "search hits" outputs in your data folder
search_dir = f"{data_dir}"
patterns = ["*.m8", "*.tsv", "*.csv", "*.txt"]
files = []
for p in patterns:
    files += glob.glob(os.path.join(search_dir, p))

# sort for readability
files = sorted(files)

print("Search-output candidates found:", len(files))
for f in files[:30]:
    print(" -", os.path.basename(f))

# 2) Try to preview a likely hits table (prefer .m8 if present)
m8_files = [f for f in files if f.endswith(".m8")]
candidate = m8_files[0] if m8_files else (files[0] if files else None)

print("\nPreview file:", candidate)

if candidate is None:
    print("No output files found in", search_dir)
else:
    # .m8 is typically tab-separated with no header; columns include evalue
    if candidate.endswith(".m8"):
        df_hits = pd.read_csv(candidate, sep="\t", header=None)
        print("Loaded .m8 shape:", df_hits.shape)
        print(df_hits.head(5))
        print("\nCommon .m8 column meaning (often):")
        print("0=query, 1=target, 2=pident, 3=alnlen, 4=mismatch, 5=gapopen, 6=qstart, 7=qend, 8=tstart, 9=tend, 10=evalue, 11=bits")
    else:
        # try tab first, then comma
        try:
            df_hits = pd.read_csv(candidate, sep="\t")
        except:
            df_hits = pd.read_csv(candidate)
        print("Loaded table shape:", df_hits.shape)
        print("Columns:", list(df_hits.columns))
        print(df_hits.head(5))


Search-output candidates found: 1
 - O43099_pssm_vs_swissprot.tsv

Preview file: /content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_pssm_vs_swissprot.tsv
Loaded table shape: (210, 9)
Columns: ['sp|O43099|PRX5_ASPFU', '1', '125', '41', '165', '1.16e-57', '179', '125.1', '100.000']
    sp|O43099|PRX5_ASPFU  1  125  41  165      1.16e-57    179  125.1  100.000
0   sp|Q5ASN8|PRX5_EMENI  1  125  41  165  1.370000e-56  176.0    125   91.200
1   sp|Q9Y8B8|PRX5_PENCI  1  125  40  164  8.980000e-55  171.0    125   83.200
2   sp|A9PCL4|PRX2_POPTR  1  125  31  159  2.200000e-52  165.0    131   37.405
3  sp|P30044|PRDX5_HUMAN  1  125  80  211  7.200000e-52  166.0    133   34.586
4  sp|Q9FR35|PRX2C_ORYSJ  2  125  32  159  8.720000e-52  163.0    130   36.923


In [197]:
import re

# 1Load model hits properly (no header)
hits = pd.read_csv(
    f"{data_dir}/O43099_pssm_vs_swissprot.tsv",
    sep="\t",
    header=None
)

# rename columns for clarity
hits.columns = [
    "target", "qstart", "qend", "tstart", "tend",
    "evalue", "bitscore", "aln_len", "pident"
]

print("Total SwissProt hits:", len(hits))

#  Apply threshold
threshold = 1e-20
filtered_hits = hits[hits["evalue"] <= threshold]

print("Hits with E-value <=", threshold, ":", len(filtered_hits))

# Extract SwissProt accession from 'sp|O43099|PRX5_ASPFU'
def extract_sp_acc(target):
    return target.split("|")[1]

filtered_hits["accession"] = filtered_hits["target"].apply(extract_sp_acc)

#Intersect with your 1001 candidates
verified = candidates[candidates["accession"].isin(filtered_hits["accession"])]

print("Verified family members after thresholding:", len(verified))
verified.head()


Total SwissProt hits: 211
Hits with E-value <= 1e-20 : 33
Verified family members after thresholding: 20


/tmp/ipython-input-451128439.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_hits["accession"] = filtered_hits["target"].apply(extract_sp_acc)


,seq_id,length,accession
0,O43099_TIR,161,O43099
1,UR50:UniRef50_O43099,161,O43099
54,UR50:UniRef50_P56577,161,P56577
70,UR50:UniRef50_P56578,161,P56578
176,UR50:UniRef50_Q69TY4,161,Q69TY4


In [198]:
#Print out the Faily-Sequence Accessions
from Bio import SeqIO

# Get verified accession set
verified_accessions = set(verified["accession"])

# Filter original family sequences
verified_sequences = [
    r for r in family_sequences
    if extract_accession(r.id) in verified_accessions
]

print("Sequences to export:", len(verified_sequences))

# Write new FASTA
verified_fasta_path = f"{data_dir}/O43099_verified_family.fasta"

with open(verified_fasta_path, "w") as out:
    SeqIO.write(verified_sequences, out, "fasta")

print("Verified family FASTA written to:")
print(verified_fasta_path)


Sequences to export: 20
Verified family FASTA written to:
/content/drive/MyDrive/BioInformatics/Biologicaldata_project/data/O43099_verified_family.fasta
